# Smart Grid Research
## "Hands-on" Simulation with mosaik

### Introduction
#### Background
Smart grids represent an evolution in electricity network management, integrating renewable energy sources, improving efficiency, and increasing the reliability of electricity distribution. Simulations play a critical role in the design, analysis, and optimization of smart grid technologies, enabling researchers and engineers to test scenarios that would be impractical or impossible to perform in real life.

#### Objective
In this notebook, we demonstrate a simulation of a smart grid ecosystem using the mosaik simulation framework. We'll simulate various components including wind turbines, power plants, batteries, and photovoltaic (PV) systems, showcasing how these elements interact within a smart grid.

#### Setting up the scenario
To get started, import the necessary libraries: 

In [ ]:
import import_ipynb

# Only for Jupyter notebooks
import nest_asyncio

nest_asyncio.apply()

import mosaik.util

### Simulation Configuration
In this section, we configure the simulators for our smart grid components. Each simulator represents a different aspect of the smart grid, from renewable energy sources to storage solutions.

In [ ]:
SIM_CONFIG = {
    'CSV': {
        'python': 'mosaik_csv:CSV',
    },
    'Wind': {
        'python': 'mosaik_components.wind:Simulator'
    },
    'PowerPlant': {
        'python': 'mosaik_components.powerplant:Simulator'
    },
    'Battery': {
        'python': 'mosaik_components.battery:Simulator'
    },
    'PV': {
        'python': 'mosaik_components.pv.pvsimulator:PVSimulator'
    },
    'Collector': {
        'python': 'simulators.collector:Collector'
    }
}

We also need to define a mosaik World object. 

In [ ]:
world = mosaik.World(SIM_CONFIG)

The simulation duration is defined to be 5 minutes in milliseconds.

In [ ]:
END = 5 * 60 * 1000 + 1

### Component Simulations
#### Wind Simulation
We simulate a wind turbine using wind speed data and a power coefficient curve. The simulation demonstrates how varying wind speeds affect power generation.

*Data Inputs*
Wind speed data (wind_data_bhv.csv) represents the varying speeds over time.
Power coefficient curve (power_coeff_curve_arek.csv) relates wind speed to power output.
Simulator Configuration
We initialize the wind simulator with the CSV file paths and configure the simulation start time and step size.

In [ ]:
# WIND
WIND_DATA = 'csv_data/wind_data_bhv.csv'
POWER_CURVE_DATA = 'csv_data/power_coeff_curve_arek.csv'
SIM_START_WIND = '2021-02-24 00:00:00'

windData = world.start("CSV",
                       sim_start=SIM_START_WIND,
                       datafile=WIND_DATA)

wind_simulator = world.start('Wind',
                             power_curve_csv=POWER_CURVE_DATA,
                             step_size=60000,
                             gen_neg=True)

#### Power Plant Simulation
Simulates both natural gas and waste power plants, demonstrating how different types of power plants can be modeled and integrated into the smart grid.

In [ ]:
# POWERPLANT
POWERPLANT_NATURALGAS_DATA = 'csv_data/powerplant_NaturalGas.csv'
SIM_START_NATURALGAS_PP = '2018-01-01 00:00:00'
POWERPLANT_WASTE_DATA = 'csv_data/powerplant_Waste.csv'
SIM_START_WASTE_PP = '2018-01-01 00:00:00'

powerplant_NaturalGas_Data = world.start("CSV",
                                         sim_start=SIM_START_NATURALGAS_PP,
                                         datafile=POWERPLANT_NATURALGAS_DATA)

powerplant_Waste_Data = world.start("CSV",
                                    sim_start=SIM_START_WASTE_PP,
                                    datafile=POWERPLANT_WASTE_DATA)

powerplant_simulator = world.start('PowerPlant',
                                   step_size=60000,
                                   number_of_schedules=5,
                                   ramp_behavior=[2, 2],
                                   min_power=0,
                                   max_power=14,
                                   interval_size=3)

#### Battery Storage
Represents a battery storage system, showcasing how energy can be stored during periods of excess generation and released during demand peaks.

In [ ]:
# BATTERY
battery_simulator = world.start('Battery',
                                step_size=60000,
                                number_of_schedules=5,
                                charge_power=11.0,
                                discharge_power=150.0,
                                battery_capacity=80,
                                start_soc=0.1,
                                interval_size=3)

#### PV Simulation
Simulates a photovoltaic system, showing how solar power can be integrated into the smart grid, varying with sunlight availability.



In [ ]:
# PV
PV_DATA = 'csv_data/pv_10kw.csv'
SIM_START = '2014-01-01 00:00:00'

pv_Data = world.start("CSV",
                      sim_start=SIM_START,
                      datafile=PV_DATA)

#### Collector
Collects and monitors data from all components, crucial for analyzing the overall performance of the smart grid simulation.

In [ ]:
# COLLECTOR
collector = world.start('Collector')

### Instantiation
Now we need to instantiate the different simulation models. 

In [ ]:
# INSTANTIATION
wind_speed = windData.Wind()
wind_sim_model = wind_simulator.WT()

powerplant_naturalgas_data = powerplant_NaturalGas_Data.NaturalGas()
powerplant_waste_data = powerplant_Waste_Data.Waste()
powerplant_sim_model = powerplant_simulator.Powerplant()

battery_sim_model = battery_simulator.Battery()

pv_data = pv_Data.PV()

monitor = collector.Monitor()

### Connections
In mosaik, we need to define connections between simulation models in order to define data flows. 

In [ ]:
# CONNECTIONS
world.connect(wind_speed, wind_sim_model, 'wind_speed')
world.connect(wind_sim_model, monitor, 'P_gen')

world.connect(powerplant_naturalgas_data, powerplant_sim_model, 'p_mw')
world.connect(powerplant_waste_data, powerplant_sim_model, 'p_mw')
world.connect(powerplant_sim_model, monitor, 'schedules')

world.connect(powerplant_sim_model, battery_sim_model, 'schedules')
world.connect(battery_sim_model, monitor, 'P_gen', 'schedules')

world.connect(pv_data, monitor, 'P[MW]')

### Execute the simulation
Finally, we execute the simulation. 

In [ ]:
world.run(until=END)

***
## Questions/Excercises
1. Analyze the dependencies between the simulation models and draw an overview that shows the simulation models and the input/output data.
2. What does the collected data contain? 
3. Add some visualization of the simulation data by modification of the collector simulator.
4. Modify the wind speed data and observe how it affects power generation.
5. Experiment with different battery capacities and analyze the impact.
6. Add a controller simulator that controls the battery and power plants to balance supply and demand.